Идеја је да се напише програм који би пратио трке за ресурсе. У програму ресурс може бити било који податак који је зачајан за приступање - читање или писање. Непожељна ситуација је да две или више нити у исто време приступе ресурсу са различитим намерама. На пример једна нит модификује податак а друга чита податак. Без одговарајуће синхронизације може се десити да у исто време прва нит модификује ресурс а друга нит чита ресурс који нема вредност коју очекујемо јер друга нити није сачекала да се заврши модификација податка. Исправност рада програма зависи од тачности редоследа извршавања нити.
У следећем коду пратимо процес трке података.
Прва класа AccessType је апстрактни домен за праћење приступања променљивим.
Друга класа VariableAccess чува информације о приступима променљивим (која
је нит и који је тип приступа).

In [1]:
class AccessType:
    READ = "READ"
    WRITE = "WRITE"

class VariableAccess:
    def __init__(self):
        self.accesses = []

    def add_access(self, thread_id, access_type):
        self.accesses.append((thread_id, access_type))

    def has_race_condition(self):
        # Detekcija trka podataka
        write_accesses = [access for access in self.accesses if access[1] == AccessType.WRITE]
        if len(write_accesses) > 1:
            return True
        read_accesses = [access for access in self.accesses if access[1] == AccessType.READ]
        if write_accesses and read_accesses:
            return True
        return False


Класа ParallelProgramAnalyzer анализира паралелне токове и детектује трке података.
У оквиру ње метода analyze_thread додаје приступе променљивим за сваку нит.
Метода detect_races проверава да ли постоје трке података на променљивим.

In [2]:
class ParallelProgramAnalyzer:
    def __init__(self):
        self.variable_accesses = {}

    def analyze_thread(self, thread_id, operations):
        for op in operations:
            var_name, access_type = op
            if var_name not in self.variable_accesses:
                self.variable_accesses[var_name] = VariableAccess()
            self.variable_accesses[var_name].add_access(thread_id, access_type)

    def detect_races(self):
        races = []
        for var_name, access in self.variable_accesses.items():
            if access.has_race_condition():
                races.append(var_name)
        return races


Покренућемо анализу и детектовати трке података.

In [3]:
# Definisanje paralelnih operacija
thread1_operations = [("x", AccessType.WRITE), ("y", AccessType.READ)]
thread2_operations = [("x", AccessType.READ), ("y", AccessType.WRITE)]

analyzer = ParallelProgramAnalyzer()
analyzer.analyze_thread("thread1", thread1_operations)
analyzer.analyze_thread("thread2", thread2_operations)

races = analyzer.detect_races()
if races:
    print(f"Detektovane trke podataka na promenljivim: {', '.join(races)}")
else:
    print("Nema detektovanih trka podataka.")


Detektovane trke podataka na promenljivim: x, y


In [4]:
import threading

x = 0
y = 0

def thread1():
    global x, y
    x += 1
    print(f"Thread 1: x = {x}")
    y += 1
    print(f"Thread 1: y = {y}")

def thread2():
    global x, y
    x += 2
    print(f"Thread 2: x = {x}")
    y += 2
    print(f"Thread 2: y = {y}")

t1 = threading.Thread(target=thread1)
t2 = threading.Thread(target=thread2)

t1.start()
t2.start()

t1.join()
t2.join()


Thread 1: x = 1
Thread 1: y = 1
Thread 2: x = 3
Thread 2: y = 3


In [5]:
thread1_operations = [("x", AccessType.WRITE), ("y", AccessType.WRITE)]
thread2_operations = [("x", AccessType.WRITE), ("y", AccessType.WRITE)]


In [6]:
analyzer = ParallelProgramAnalyzer()
analyzer.analyze_thread("thread1", thread1_operations)
analyzer.analyze_thread("thread2", thread2_operations)

races = analyzer.detect_races()
if races:
    print(f"Detektovane trke podataka na promenljivim: {', '.join(races)}")
else:
    print("Nema detektovanih trka podataka.")


Detektovane trke podataka na promenljivim: x, y


Цео код.

In [10]:
import threading

class AccessType:
    READ = "READ"
    WRITE = "WRITE"

class VariableAccess:
    def __init__(self):
        self.accesses = []

    def add_access(self, thread_id, access_type):
        self.accesses.append((thread_id, access_type))

    def has_race_condition(self):
        write_accesses = [access for access in self.accesses if access[1] == AccessType.WRITE]
        if len(write_accesses) > 1:
            return True
        read_accesses = [access for access in self.accesses if access[1] == AccessType.READ]
        if write_accesses and read_accesses:
            return True
        return False

class ParallelProgramAnalyzer:
    def __init__(self):
        self.variable_accesses = {}

    def analyze_thread(self, thread_id, operations):
        for op in operations:
            var_name, access_type = op
            if var_name not in self.variable_accesses:
                self.variable_accesses[var_name] = VariableAccess()
            self.variable_accesses[var_name].add_access(thread_id, access_type)

    def detect_races(self):
        races = []
        for var_name, access in self.variable_accesses.items():
            if access.has_race_condition():
                races.append(var_name)
        return races

# Definisanje paralelnih operacija
thread1_operations = [("x", AccessType.WRITE), ("y", AccessType.WRITE)]
thread2_operations = [("x", AccessType.WRITE), ("y", AccessType.WRITE)]

analyzer = ParallelProgramAnalyzer()
analyzer.analyze_thread("thread1", thread1_operations)
analyzer.analyze_thread("thread2", thread2_operations)

races = analyzer.detect_races()
if races:
    print(f"Detektovane trke podataka na promenljivim: {', '.join(races)}")
else:
    print("Nema detektovanih trka podataka.")

# Paralelni program
x = 0
y = 0

def thread1():
    global x, y
    x += 1
    print(f"Thread 1: x = {x}")
    y += 1
    print(f"Thread 1: y = {y}")

def thread2():
    global x, y
    x += 2
    print(f"Thread 2: x = {x}")
    y += 2
    print(f"Thread 2: y = {y}")

t1 = threading.Thread(target=thread1)
t2 = threading.Thread(target=thread2)

t1.start()
t2.start()

t1.join()
t2.join()


Detektovane trke podataka na promenljivim: x, y
Thread 1: x = 1
Thread 1: y = 1
Thread 2: x = 3
Thread 2: y = 3


НАПОМЕНА - биће додати кодови за апстрактне домене 